<a href="https://colab.research.google.com/github/STPREETHI/git-scm-demo/blob/main/EX_5_and_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# ------------------------------------
# GET IMAGE FROM USER (JUPYTER / COLAB)
# ------------------------------------
from google.colab import files
uploaded = files.upload()   # User uploads image(s)

# Get the first uploaded file name
image_path = list(uploaded.keys())[0]

# Read image
color = cv2.imread(image_path)
gray = cv2.cvtColor(color, cv2.COLOR_BGR2GRAY)

# ------------------------------------
# 1. GRAYSCALE HISTOGRAM EQUALIZATION
# ------------------------------------
gray_eq = cv2.equalizeHist(gray)

# ------------------------------------
# 2. COLOR HISTOGRAM EQUALIZATION
# ------------------------------------

# RGB SPACE
rgb = cv2.cvtColor(color, cv2.COLOR_BGR2RGB)
r, g, b = cv2.split(rgb)
rgb_eq = cv2.merge((
    cv2.equalizeHist(r),
    cv2.equalizeHist(g),
    cv2.equalizeHist(b)
))

# HSV SPACE
hsv = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
h, s, v = cv2.split(hsv)
hsv_eq = cv2.merge((h, s, cv2.equalizeHist(v)))
hsv_eq = cv2.cvtColor(hsv_eq, cv2.COLOR_HSV2RGB)

# YCrCb SPACE
ycrcb = cv2.cvtColor(color, cv2.COLOR_BGR2YCrCb)
y, cr, cb = cv2.split(ycrcb)
ycrcb_eq = cv2.merge((cv2.equalizeHist(y), cr, cb))
ycrcb_eq = cv2.cvtColor(ycrcb_eq, cv2.COLOR_YCrCb2RGB)

# ------------------------------------
# 3. HISTOGRAM MATCHING
# (Using blurred version as reference)
# ------------------------------------
reference = cv2.GaussianBlur(gray, (31, 31), 0)

def histogram_matching(src, ref):
    src_flat = src.ravel()
    ref_flat = ref.ravel()

    s_vals, s_counts = np.unique(src_flat, return_counts=True)
    r_vals, r_counts = np.unique(ref_flat, return_counts=True)

    s_cdf = np.cumsum(s_counts) / src_flat.size
    r_cdf = np.cumsum(r_counts) / ref_flat.size

    interp = np.interp(s_cdf, r_cdf, r_vals)
    matched = interp[np.searchsorted(s_vals, src_flat)]

    return matched.reshape(src.shape)

matched = histogram_matching(gray, reference)

# ------------------------------------
# 4. CLAHE vs GLOBAL HISTOGRAM EQUALIZATION
# ------------------------------------
global_eq = cv2.equalizeHist(gray)
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
clahe_img = clahe.apply(gray)

# ------------------------------------
# DISPLAY RESULTS
# ------------------------------------
plt.figure(figsize=(15,10))

plt.subplot(3,4,1); plt.title("Original Gray"); plt.imshow(gray, cmap='gray'); plt.axis('off')
plt.subplot(3,4,2); plt.title("Gray HE"); plt.imshow(gray_eq, cmap='gray'); plt.axis('off')
plt.subplot(3,4,3); plt.title("Global HE"); plt.imshow(global_eq, cmap='gray'); plt.axis('off')
plt.subplot(3,4,4); plt.title("CLAHE"); plt.imshow(clahe_img, cmap='gray'); plt.axis('off')

plt.subplot(3,4,5); plt.title("Original RGB"); plt.imshow(rgb); plt.axis('off')
plt.subplot(3,4,6); plt.title("RGB HE"); plt.imshow(rgb_eq); plt.axis('off')
plt.subplot(3,4,7); plt.title("HSV HE"); plt.imshow(hsv_eq); plt.axis('off')
plt.subplot(3,4,8); plt.title("YCrCb HE"); plt.imshow(ycrcb_eq); plt.axis('off')

plt.subplot(3,4,9); plt.title("Source"); plt.imshow(gray, cmap='gray'); plt.axis('off')
plt.subplot(3,4,10); plt.title("Reference"); plt.imshow(reference, cmap='gray'); plt.axis('off')
plt.subplot(3,4,11); plt.title("Histogram Matched"); plt.imshow(matched, cmap='gray'); plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist

# -----------------------------
# LOAD DATA
# -----------------------------
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# -----------------------------
# PREPROCESSING
# -----------------------------
def normalize(images):
    return images.astype("float32") / 255.0

def histogram_equalization(images):
    eq_images = []
    for img in images:
        img_uint8 = (img * 255).astype(np.uint8)
        eq = cv2.equalizeHist(img_uint8)
        eq_images.append(eq / 255.0)
    return np.array(eq_images)

x_train_norm = normalize(x_train)
x_test_norm = normalize(x_test)

x_train_eq = histogram_equalization(x_train_norm)
x_test_eq = histogram_equalization(x_test_norm)

# Add channel dimension
x_train_norm = x_train_norm[..., np.newaxis]
x_test_norm = x_test_norm[..., np.newaxis]
x_train_eq = x_train_eq[..., np.newaxis]
x_test_eq = x_test_eq[..., np.newaxis]

# -----------------------------
# SIMPLE CNN (LOW CAPACITY)
# -----------------------------
def build_simple_cnn():
    model = models.Sequential([
        layers.Conv2D(16, (3,3), activation='relu', input_shape=(28,28,1)),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.4),

        layers.Conv2D(32, (3,3), activation='relu'),
        layers.MaxPooling2D((2,2)),
        layers.Dropout(0.4),

        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(10, activation='softmax')
    ])

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

# -----------------------------
# TRAIN WITHOUT HISTOGRAM EQ
# -----------------------------
model_no_eq = build_simple_cnn()
model_no_eq.fit(
    x_train_norm, y_train,
    epochs=5,
    batch_size=256,
    validation_split=0.1,
    verbose=0
)

_, acc_no_eq = model_no_eq.evaluate(x_test_norm, y_test, verbose=0)

# -----------------------------
# TRAIN WITH HISTOGRAM EQ
# -----------------------------
model_eq = build_simple_cnn()
model_eq.fit(
    x_train_eq, y_train,
    epochs=5,
    batch_size=256,
    validation_split=0.1,
    verbose=0
)

_, acc_eq = model_eq.evaluate(x_test_eq, y_test, verbose=0)

# -----------------------------
# RESULTS
# -----------------------------
print(f"Accuracy WITHOUT Histogram Equalization: {acc_no_eq*100:.2f}%")
print(f"Accuracy WITH Histogram Equalization: {acc_eq*100:.2f}%")
